# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
!pip install langchain langchain-openai python-dotenv

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 1.7 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import userdata
import os
# Retrieve the API key from Colab secrets
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# Set it as an environment variable, which LangChain often uses by default
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
print("OpenAI API key loaded and set as environment variable.")

OpenAI API key loaded and set as environment variable.


In [ ]:
#!pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv("/Data.csv")

In [ ]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain


In [8]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


In [9]:
prompt = ChatPromptTemplate.from_template( "Tell me about the product: {product_description}"
)

In [10]:

chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [11]:
product = "Luxury Air Mattress"
chain.run(product)

/tmp/ipython-input-1892552926.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'A luxury air mattress is a high-quality, durable mattress that provides a comfortable and supportive sleeping surface. These mattresses are typically made with advanced materials and construction techniques to ensure optimal comfort and durability.\n\nLuxury air mattresses often feature multiple air chambers that can be adjusted to customize the firmness level to suit individual preferences. Many models also include additional features such as built-in pumps for easy inflation and deflation, plush pillow tops for extra comfort, and antimicrobial treatments to prevent mold and mildew growth.\n\nThese mattresses are ideal for use at home as a permanent bed or as a temporary sleeping solution for guests. They are also popular for outdoor use, such as camping or RV trips, as they are portable and easy to set up.\n\nOverall, a luxury air mattress offers a luxurious sleeping experience with the convenience and versatility of an inflatable bed.'

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
        model="gpt-3.5-turbo", temperature=0.9)
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "As an expert in product marketing, analyze the following product description:\n\n{product_description}\n\n"
    "Based on this, suggest 3 concise, catchy, and benefit-oriented taglines. "
    "Format them as a numbered list."
)
# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on these taglines for a product:\n\n{taglines_from_first_chain}\n\n"
    "Suggest a creative, short social media post (e.g., for Instagram or X) "
    "that incorporates one of these taglines. Include relevant emojis and hashtags."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
1. "Experience the Ultimate Comfort: Luxury Air Mattress"
2. "Elevate Your Sleep: Luxury Air Mattress"
3. "Rest in Style: Luxury Air Mattress"
"Upgrade your sleep game with our new Luxury Air Mattress! 💤✨ Experience the ultimate comfort and elevate your rest to a whole new level. #LuxurySleep #ComfyNights"

> Finished chain.


'"Upgrade your sleep game with our new Luxury Air Mattress! 💤✨ Experience the ultimate comfort and elevate your rest to a whole new level. #LuxurySleep #ComfyNights"'

**Repeat the above twice for different products**

## SequentialChain

In [33]:
from langchain.chains import SequentialChain

In [63]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "As experienced product testerr, give me a summary of the best reviews for the following product:\n\n{product_description}\n\n"
  "The summary has 2 sentences max.\n"
  "Based on this, tell me 1 really cool fact about this product.\n\n"

)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key= "Recommendation"
                    )


In [64]:
second_prompt = ChatPromptTemplate.from_template(
    "You are social media manager.\n"
    "This is my product recommendation:\n{Recommendation}\n\n"
    "Give me three ideas, how to hype this on Instagram.\n"


)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="On_Social_Media"
                    )


In [65]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to German:\n\n"
    "{On_Social_Media}\n\nGerman Translation:"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="Translation"
                      )


In [66]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Write a friendly and professional follow-up message to a customer based on the summary and translation review."
        "this recommendation and these marketing ideas.n\n"
        "Recommendation:\n{Recommendation}\n\n"
        "Social media ideas:\n{On_Social_Media}\n\n"
        "German version:\n{Translation}\n\n"
        "Follow-up message:"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="Follow_up"
                     )


In [67]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["product_description"],
    output_variables=["Recommendation", "On_Social_Media", "Translation", "Follow_up"],
    verbose=True
)

In [68]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'product_description': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'Recommendation': 'The product received negative reviews regarding its taste and foam quality, with some speculating it may be an old batch or counterfeit. \n\nOne cool fact about this product is that it is commonly found in stores and has a reputation for having a much better taste than what was experienced by the reviewer.',
 'On_Social_Media': '1. Host a giveaway: Create a visually appealing post showcasing the product and its positive attributes, while also addressing the negative reviews in a transparent and honest manner. Encourage followers to participate in a giveaway where they can win a free sample of the product to try for themselves.\n\n2. Collaborate with influencers: Partner with popular food and beverage influencers who can create visually engaging content featuring the product. H

**Repeat the above twice for different products or reviews**

## Router Chain

In [39]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [40]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [41]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [42]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [43]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [44]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [45]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [46]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipython-input-3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [47]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which absorbs all incident electromagnetic radiation and emits radiation in all directions. The spectral distribution of black body radiation follows Planck's law, which describes how the intensity of radiation emitted by a black body at a given wavelength or frequency is dependent on its temperature. This phenomenon is important in understanding the behavior of objects at high temperatures, such as stars, and is a key concept in the field of thermal physics."

In [48]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 equals 4.'

In [49]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for building and regulating all the structures and functions of a living organism. DNA is essential for the growth, development, and maintenance of an organism. It contains the information needed to produce proteins, which are the building blocks of cells and perform a wide variety of functions in the body.\n\nAdditionally, DNA contains the genetic code that determines an organism's traits, such as hair color, eye color, and susceptibility to certain diseases. Each cell in our body contains the same DNA, with the exception of sperm and egg cells, which have half the amount of DNA to combine during fertilization to create a new organism.\n\nOverall, DNA is crucial for the proper functioning of cells and the continuation of life processes, which is why every cell in our body contains DNA."

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [69]:
chain.run("How much blood does a woman have normally?")



> Entering new MultiPromptChain chain...
biology: {'input': 'How much blood does a woman normally have?'}
> Finished chain.


'On average, a woman typically has about 4 to 5 liters of blood in her body. This amount can vary depending on factors such as body size, age, and overall health. Blood volume can also fluctuate during pregnancy or menstruation.'

In [74]:
chain.run("Why do so many people between ages 40-60 get knee problems? How do I prevent that happening too early?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why do so many people between ages 40-60 get knee problems? How do I prevent that happening too early?'}
> Finished chain.


'As a biologist, I can provide insights into why knee problems are common among individuals aged 40-60 and suggest ways to prevent them from occurring prematurely.\n\nOne of the main reasons why knee problems are prevalent in this age group is due to the natural aging process. As we get older, the cartilage in our joints, including the knee, wears down, leading to conditions such as osteoarthritis. Additionally, factors such as previous injuries, obesity, genetics, and a sedentary lifestyle can also contribute to an increased risk of knee problems.\n\nTo prevent knee problems from occurring too early, it is important to take care of your knee health throughout your life. This includes:\n\n1. Maintaining a healthy weight: Excess weight puts extra stress on your knees, increasing the risk of developing knee problems. Maintaining a healthy weight through a balanced diet and regular exercise can help reduce this risk.\n\n2. Staying active: Regular exercise, particularly activities that str

In [75]:
chain.run("Do aliens exist?")



> Entering new MultiPromptChain chain...
None: {'input': 'Do aliens exist?'}
> Finished chain.


'There is currently no conclusive evidence that proves the existence of aliens. While many scientists believe that it is statistically likely that intelligent life exists elsewhere in the universe given the vast number of planets and galaxies, no direct evidence of extraterrestrial life has been found. The search for extraterrestrial life continues through programs like SETI (Search for Extraterrestrial Intelligence) and missions to Mars and other planets that could potentially support life.'

In [77]:
chain.run("What age am I?")



> Entering new MultiPromptChain chain...
None: {'input': 'What age am I?'}
> Finished chain.


"I'm sorry, I cannot determine your age as I do not have access to personal information."